In [18]:
%matplotlib inline
import pandas as pd
import numpy as np
import os
# Import the main functionality from the SimFin Python API.
import simfin as sf

# Import names used for easy access to SimFin's data-columns.
from simfin.names import *

import yfinance as yf
import yahoo_fin.stock_info as si
# import pandas_datareader.data as web

from dateutil.relativedelta import relativedelta
from datetime import datetime

pd.set_option('display.max_rows', None, 'display.max_columns', None)

## Model 1 
#### Target T0_5>0 --> Event 
#### Target T0_5<0 --> Non-event

In [19]:
dfmdl1=pd.read_csv('data/model_1.csv')

In [20]:
dfmdl1.shape

(8259, 84)

In [21]:
dfmdl1=dfmdl1.dropna(subset=['T0_5'])
dfmdl1.shape

(7841, 84)

In [22]:
dfmdl1['CAPX'] = dfmdl1['CAPX'].fillna(0)
dfmdl1['Dividends'] = dfmdl1['Dividends'].fillna(0)
dfmdl1['Dividend_yields_new'] = dfmdl1['Dividend_yields_new'].fillna(0)
dfmdl1['Dividend_yields_new'] = dfmdl1['Dividend_yields_new'].fillna(0)

In [23]:
dfmdl1.isna().sum()

Ticker                     0
beta_30avg                12
adjclose                  12
Fiscal Year                0
Fiscal Period              0
Revenue                    4
Net_income                 0
Shares                     0
Cash                       5
Assets                     0
CFO                        0
CAPX                       0
Dividends                  0
FCF                        0
EPSsuprise_ind             0
Revenue_qoq              414
Revenue_yoy             1724
Net_Income_qoq           414
Net_Income_yoy          1724
Gross_margin             386
Gross_margin_qoq         789
Gross_margin_yoy        2048
Operating_margin           4
Operating_margin_qoq     420
Operating_margin_yoy    1732
Net_margin                 4
Net_margin_qoq           420
Net_margin_yoy          1732
Cash_yoy                1724
Assets_yoy              1724
Leverage                   0
CFO_qoq                  414
FCF_qoq                  414
Sector                     0
Industry      

In [16]:
dfmdl1['target'] = np.where(dfmdl1['T0_5']>0, 1, 0)

In [24]:
dfmdl1.shape

(7841, 84)

In [26]:
dfmdl1_bin=dfmdl1[['Ticker','Target','Next_EarningDate',]]
var='beta_30avg'
dfmdl1['qbin_'+var] = pd.qcut(dfmdl1[var], q=10, precision=0)

In [27]:
dfmdl1.head()

,Ticker,beta_30avg,adjclose,Fiscal Year,Fiscal Period,Revenue,Net_income,Shares,Cash,Assets,CFO,CAPX,Dividends,FCF,EPSsuprise_ind,Revenue_qoq,Revenue_yoy,Net_Income_qoq,Net_Income_yoy,Gross_margin,Gross_margin_qoq,Gross_margin_yoy,Operating_margin,Operating_margin_qoq,Operating_margin_yoy,Net_margin,Net_margin_qoq,Net_margin_yoy,Cash_yoy,Assets_yoy,Leverage,CFO_qoq,FCF_qoq,Sector,Industry,Next_EarningDate,RSI,MFI,adjclose_SMA_12,adjclose_SMA_30,adjclose_EWMA_12,adjclose_EWMA_30,adjclose_UpperBand,adjclose_LowerBand,SMA_12_SMA_30,EWMA_12_EWMA_30,UpperBand_LowerBand,^GSPC,^GSPC_pctchange_5,^GSPC_pctchange_21,^GSPC_pctchange_yoy,^RUT,^RUT_pctchange_5,^RUT_pctchange_21,^RUT_pctchange_yoy,^VIX,^VIX_pctchange_5,^VIX_pctchange_21,^VIX_pctchange_yoy,GC=F,GC=F_pctchange_5,GC=F_pctchange_21,GC=F_pctchange_yoy,CL=F,CL=F_pctchange_5,CL=F_pctchange_21,CL=F_pctchange_yoy,^TNX,^TNX_pctchange_5,^TNX_pctchange_21,^TNX_pctchange_yoy,^FVX,^FVX_pctchange_5,^FVX_pctchange_21,^FVX_pctchange_yoy,cs_last5,T0,sameday_jump,T0_5,PE_new,PS_new,PB_new,Dividend_yields_new,Market_cap_mm,qbin_beta_30avg
0,A,1.562381,46.201164,2016,Q3,1.044000e+09,124000000,325000000.0,2.199000e+09,7734000000,194000000.0,0.0,-37000000.0,60000000,True,NaN,NaN,NaN,NaN,0.519157,NaN,NaN,0.139847,NaN,NaN,0.118774,NaN,NaN,NaN,NaN,2.282763,NaN,NaN,Healthcare,Diagnostics & Research,2016-11-15,50.848450,49.160386,1.009450,1.002813,0.957853,0.957749,0.946279,1.066530,0.993425,0.999892,1.127077,2180.39,1.91,2.53,6.34,1302.14,8.95,7.60,12.91,13.37,-28.66,-17.52,-29.03,1224.0,-4.25,-2.32,14.53,45.81,2.05,-9.02,12.64,2.24,20.19,28.11,12.80,1.67,25.77,34.67,18.14,-0.085933,43.418201,-0.042175,-0.020705,113.797705,13.516203,1.824530,-0.002622,14110.915470,"(1.4, 1.6]"
1,A,1.727714,43.418201,2016,Q4,1.111000e+09,126000000,324000000.0,2.289000e+09,7794000000,234000000.0,0.0,-38000000.0,90000000,True,0.064176,NaN,0.016129,NaN,0.529253,0.010096,NaN,0.164716,0.024870,NaN,0.113411,-0.005363,NaN,NaN,NaN,2.196731,0.206186,0.500000,Healthcare,Diagnostics & Research,2017-02-14,65.809029,63.517597,1.016679,1.037272,0.972830,0.993873,0.991504,1.087470,1.020256,1.021630,1.096788,2337.58,1.94,2.77,21.32,1396.65,2.61,1.79,38.13,10.74,-4.87,-4.36,-51.86,1223.9,-0.83,2.39,-1.23,53.20,1.97,1.58,80.71,2.47,5.06,0.12,40.98,1.96,7.93,1.03,62.12,0.225033,48.329895,-0.038020,0.018853,124.276873,14.094407,2.009095,-0.002427,15658.885986,"(1.6, 6.2]"
2,A,2.014411,48.329895,2017,Q1,1.067000e+09,168000000,322000000.0,2.241000e+09,7872000000,116000000.0,0.0,-42000000.0,-48000000,True,-0.039604,NaN,0.333333,NaN,0.537957,0.008704,NaN,0.193065,0.028348,NaN,0.157451,0.044039,NaN,NaN,NaN,2.203807,-0.504274,-1.533333,Healthcare,Diagnostics & Research,2017-05-22,57.895288,66.358370,0.995724,1.018778,0.961621,0.976073,0.965030,1.078866,1.023153,1.015029,1.117962,2394.02,-0.35,1.93,16.89,1377.14,-1.20,-0.20,23.91,10.93,4.89,-25.29,-30.91,1260.7,2.61,-2.07,0.52,50.73,3.85,2.24,5.34,2.25,-3.22,-1.83,43.57,1.80,-2.86,-1.21,60.45,0.091067,53.920761,-0.045988,0.076498,103.348125,16.272245,2.205600,-0.002419,17362.485077,"(1.6, 6.2]"
3,A,1.260812,53.920761,2017,Q2,1.102000e+09,164000000,321000000.0,2.389000e+09,8016000000,257000000.0,0.0,-43000000.0,148000000,True,0.032802,NaN,-0.023810,NaN,0.537205,-0.000752,NaN,0.182396,-0.010669,NaN,0.148820,-0.008630,NaN,NaN,NaN,2.201593,1.215517,-4.083333,Healthcare,Diagnostics & Research,2017-08-15,45.332843,46.147494,0.992396,0.985299,0.957375,0.954990,0.959937,1.012037,0.992848,0.997510,1.054274,2464.61,-0.42,0.22,13.15,1383.24,-1.91,-3.38,12.36,12.04,9.85,22.61,-4.75,1273.7,1.38,3.32,-4.65,47.55,-3.29,3.32,6.88,2.27,1.12,1.52,26.31,1.82,1.28,1.11,41.97,0.237633,57.220871,-0.041526,0.060290,111.999388,16.667785,2.291405,-0.002341,18367.899582,"(1.2, 1.4]"
4,A,1.045623,57.220871,2017,Q3,1.114000e+09,175000000,321000000.0,2.563000e+09,8261000000,228000000.0,0.0,-42000000.0,174000000,True,0.010889,0.06705,0.067073,0.41129,0.535009,-0.002196,0.015852,0.180431,-0.001965,0.040584,0.157092,0.008271,